In [9]:
import os
from dotenv import load_dotenv
load_dotenv()

import mlflow
os.environ["MLFLOW_TRACKING_URI"] = "databricks"
mlflow.set_tracking_uri("databricks")

print("DATABRICKS_HOST =", os.getenv("DATABRICKS_HOST"))
print("TOKEN definido?  ", "Sí" if os.getenv("DATABRICKS_TOKEN") else "No")
print("MLflow Tracking URI:", mlflow.get_tracking_uri())


DATABRICKS_HOST = None
TOKEN definido?   Sí
MLflow Tracking URI: databricks


In [13]:
import os
from dotenv import load_dotenv
load_dotenv(override=True)

print("HOST:", os.getenv("DATABRICKS_HOST"))
print("TOKEN definido?:", "Sí" if os.getenv("DATABRICKS_TOKEN") else "No")


HOST: None
TOKEN definido?: Sí


In [7]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split

# dataset mini: columnas similares a las que usas
N = 5000
rng = np.random.default_rng(42)
df = pd.DataFrame({
    "PULocationID": rng.integers(1, 50, size=N),
    "DOLocationID": rng.integers(1, 50, size=N),
    "trip_distance": rng.gamma(2.0, 1.5, size=N),
})

# target (duration) con algo de señal
df["duration"] = 5 + 2.5*df["trip_distance"] + rng.normal(0, 2.0, size=N)

# split
df_train, df_val = train_test_split(df, test_size=0.2, random_state=42)

# features como en tu notebook (PU_DO categórico + numérico)
df_train = df_train.copy()
df_val   = df_val.copy()
df_train["PU_DO"] = df_train["PULocationID"].astype(str) + "_" + df_train["DOLocationID"].astype(str)
df_val["PU_DO"]   = df_val["PULocationID"].astype(str)   + "_" + df_val["DOLocationID"].astype(str)

categorical = ["PU_DO"]
numerical   = ["trip_distance"]

dv = DictVectorizer()
train_dicts = df_train[categorical + numerical].to_dict(orient="records")
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient="records")
X_val = dv.transform(val_dicts)

y_train = df_train["duration"].values
y_val   = df_val["duration"].values

X_train.shape, X_val.shape


((4000, 1922), (1000, 1922))